In [175]:
import duckdb
from pathlib import Path
# import pandas as pd

Task 1 - EDA in python

In [145]:
# duckdb database path
duckdb_path = "data/sakila.duckdb"      # sets the path for the DuckDB database file
Path(duckdb_path).unlink(missing_ok=True)  # deletes the duckdb file if it already exists

# Ingest Sakila dataset into DuckDB
with duckdb.connect(duckdb_path) as conn, open("sql/load_sakila.sql") as ingest_scripts:  # opens a Duckdb connection and the SQL script file
    conn.sql(ingest_scripts.read()) # 


In [176]:

# read all tables into pandas DataFrames
with duckdb.connect(duckdb_path) as conn:
    table_names = conn.sql("SHOW TABLES").df()["name"].tolist()
    dfs = {table_name: conn.sql(f"SELECT * FROM {table_name}").df() for table_name in table_names}

dfs ["film"].head()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,<NA>,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2021-03-06 15:52:00
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,<NA>,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2021-03-06 15:52:00
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,<NA>,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2021-03-06 15:52:00
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,<NA>,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2021-03-06 15:52:00
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,<NA>,6,2.99,130,22.99,G,Deleted Scenes,2021-03-06 15:52:00


a) Which movies are longer than 3 hours (180 minutes), show the title and its length?

In [177]:
# film longer than 3 hours
film_long = duckdb.sql("""
    SELECT title, length
    FROM film
    WHERE length > 180
""").df()
film_long.head()

,title,length
0,ANALYZE HOOSIERS,181
1,BAKED CLEOPATRA,182
2,CATCH AMISTAD,183
3,CHICAGO NORTH,185
4,CONSPIRACY SPIRIT,184


In [179]:
len(film_long)

39

In [180]:

# Count the number of films in the film table
duckdb.sql( """SELECT * FROM film """).df()

,title,length
0,ANALYZE HOOSIERS,181
1,BAKED CLEOPATRA,182
2,CATCH AMISTAD,183
3,CHICAGO NORTH,185
4,CONSPIRACY SPIRIT,184
5,CONTROL ANTHEM,185
6,CRYSTAL BREAKING,184
7,DARN FORRESTER,185
8,FRONTIER CABIN,183
9,GANGS PRIDE,185


b) Which movies have the word "love" in its title? Show the following columns
title
rating
length
description

In [193]:

love_films = duckdb.sql("""
    SELECT "title", "length", "rating", "description" 
    FROM film
    WHERE title LIKE '%love%'
""").df()

films.columns

BinderException: Binder Error: Referenced column "rating" not found in FROM clause!
Candidate bindings: "title", "length"

In [174]:

films_love = duckdb.sql("""
    DESC
    SELECT title, length
    FROM film
    WHERE title LIKE '%love%';

""").df()

films_love.head()

,column_name,column_type,null,key,default,extra
0,title,VARCHAR,YES,None,None,None
1,length,BIGINT,YES,None,None,None


In [167]:

dfs["film"].columns

Index(['film_id', 'title', 'description', 'release_year', 'language_id',
       'original_language_id', 'rental_duration', 'rental_rate', 'length',
       'replacement_cost', 'rating', 'special_features', 'last_update'],
      dtype='object')

c) Calculate descriptive statistics on the length column, The Manager wants, shortest, average, median and longest movie length

In [205]:
length_statistics = duckdb.sql("""
    SELECT
        MIN(length) AS shortest_length,
        ROUND(AVG(length)) AS average_length,
        MEDIAN(length) AS median_length,
        MAX(length) AS longest_length
    FROM film
""").df()

length_statistics.head()


,shortest_length,average_length,median_length,longest_length
0,181,183.0,183.0,185


d) The 10 most expensive movies to rent per day.

In [ ]:

most_expensive_rentals = duckdb.sql(""" 
    SELECT title, 
            rental_rate, 
            rental_duration,
            (rental_rate / rental_duration) AS cost_per_day # calculate cost per day
    FROM film
    ORDER BY cost_per_day ASC # ASC for lowest to highest
    LIMIT 10
""").df() 

most_expensive_rentals.head()

e) The top 10 actors with the number of movies they have
played in.

In [ ]:
top_actors = duckdb.sql(""" 
    SELECT a.first_name, 
           a.last_name
        FROM actor
           """)

top_actors.head()

In [ ]:
duckdb.sql("""
    SELECT 
        actor,
        COUNT (*) AS number_film
    FROM film
    GROUP BY actor
    ORDER BY number_of_movies DESC
    LIMIT 10
""").df()



# F) Write down 3-5 questions

1. What are the 5 longest movies

In [ ]:
longest_films = duckdb.sql("""
    SELECT title, length
    FROM film
    ORDER BY length DESC  ###################
    LIMIT 5
""").df()

longest_films.head()

Select movies starting with the letter A

In [243]:
film_a = duckdb.sql("""
    SELECT title
    FROM film
    Where title LIKE 'A%'
    order BY title ASC
""").df()

film_a.head()

,title
0,ANALYZE HOOSIERS
